In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/matriculas/gt.txt
/kaggle/input/matriculas/images/1f855390ac408473250da4281ddecbfa9888a8614d50f5d69de09afc6f548c27_1.jpg
/kaggle/input/matriculas/images/1f7780a0132588c410678afe1c4e9bdad9a6aa5d35a4cdb47a57e977b56bdfdc_1.jpg
/kaggle/input/matriculas/images/2dc487a1f375370bd9af2448a2a4bef19e69a6a1ab8ff54b88e681155b2c2edb_1.jpg
/kaggle/input/matriculas/images/1de0301fcb918e5ee9f020cc343a430a89e80afde2b8a931bdd1e1f40bcb642a_1.jpg
/kaggle/input/matriculas/images/0aff94b0cc4cfef9f1c7b1aca3be6b5d10b2528f1a2310306a185124ef3043cc_1.jpg
/kaggle/input/matriculas/images/0e1e9fd3b5bfe7a712de9a2c526c03e3135a3aa3906294e1bae26bfbac52886d_1.jpg
/kaggle/input/matriculas/images/4b334e9707ed29695f8eccd4525ae7956c353f5aa130347e10e0af621022e066_1.jpg
/kaggle/input/matriculas/images/1f30d2f1a6dd81c2aa3405ba3479f365d6e0a8d8436aa28fa4bb4ae9421a3d0c_1.jpg
/kaggle/input/matriculas/images/0a7b57cd65e08970cacc537a6becba9e59cd91d63c41cc65913f0ca029e0a230_1.jpg
/kaggle/input/matriculas/images/2e208ecea

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import cv2

import re
import math
from glob import glob
from tqdm import tqdm
import random
from datetime import datetime

In [3]:
DATASET_STEM = "matriculas" # <--- EDITA AQUÍ
DATASET_PATH = f"/kaggle/input/{DATASET_STEM}"
IMG_DIR = os.path.join(DATASET_PATH, "images")
GT_FILE = os.path.join(DATASET_PATH, "gt.txt")


OUTPUT_MODEL = "/kaggle/working/ocr_ctc.pt"
BATCH_SIZE = 32
NUM_EPOCHS = 30
LR = 1e-3
IMG_W, IMG_H = 128, 32 # ancho x alto para CRNN (ancho grande, alto pequeño)


# Caracteres permitidos: blank en el índice 0
CHARS = "-0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ" # '-' = blank
char2idx = {c: i for i, c in enumerate(CHARS)}
idx2char = {i: c for i, c in enumerate(CHARS)}
NUM_CLASSES = len(CHARS)

In [4]:
assert os.path.exists(IMG_DIR), f"No existe {IMG_DIR}. Asegúrate de subir las imágenes al dataset."
assert os.path.exists(GT_FILE), f"No existe {GT_FILE}. Asegúrate de que gt.txt esté en el dataset."


raw_lines = []
with open(GT_FILE, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if line:
            raw_lines.append(line)


clean_samples = []


# Regex para matrículas típicas: 4 dígitos opcionales + letras/dígitos (flexible)
# Ajusta si tus matrículas tienen otro formato
plate_regex = re.compile(r"([0-9]{3,4}\s*[A-Z0-9]{2,6})")


for line in raw_lines:
    # primer split por coma (nombre de archivo, resto)
    parts = line.split(",", 1)
    if len(parts) == 1:
        # intenta espacio separado
        parts = line.split(maxsplit=1)
        if len(parts) == 1:
            continue
    filename = parts[0].strip().strip('"')
    rest = parts[1].strip()


    # extraer todas las coincidencias de matrícula
    matches = plate_regex.findall(rest)
    if not matches:
        # última oportunidad: todo lo que está entre comillas
        quoted = re.findall(r'"([^"]+)"', rest)
        if quoted:
            # dividir por comillas dobles pegadas
            for q in quoted:
                q = q.replace('""', '"')
                m = plate_regex.findall(q)
                if m:
                    for mm in m:
                        clean_samples.append((filename, mm.replace(" ", "")))
            continue
        else:
            # intentar tomar todo el resto como label
            lab = rest.replace('"', '').strip()
            if lab:
                clean_samples.append((filename, lab.replace(" ", "")))
            continue
    
    
    for m in matches:
        clean_samples.append((filename, m.replace(" ", "")))


# Filtrar por existencia de archivo
clean_filtered = []
missing = []
for fn, txt in clean_samples:
    p = os.path.join(IMG_DIR, fn)
    if os.path.exists(p):
        clean_filtered.append((fn, txt))
    else:
        missing.append(fn)


print(f"Total raw lines: {len(raw_lines)}")
print(f"Total extracted samples: {len(clean_samples)}")
print(f"Total samples with images: {len(clean_filtered)}")
print(f"Total missing files: {len(set(missing))}")
if len(set(missing)) > 0:
    print("Ejemplos missing:", list(set(missing))[:10])


# Crear DataFrame final
df = pd.DataFrame(clean_filtered, columns=["filename", "text"]).drop_duplicates().reset_index(drop=True)
print(df.head(10))

Total raw lines: 66
Total extracted samples: 72
Total samples with images: 72
Total missing files: 0
                                            filename     text
0  1f46e58e6c5c2db36cf230aa1137a69cea4a1825f5f969...  3654MYH
1  1dbef2f4339284152a2eb1af833b9d699b67d5c8451da5...  0624LXM
2  1d18066355939f6597dd56962339cfce5649c4afee1345...  4517MFC
3  1d0842d89fb36e8191c2f3ae595f0a94996b0343efec2e...  3561CXR
4  1c4b2371b3b8b6db73b450b869a570239b659fd64e7e1e...  0576MSH
5  1b958e7132e9b455f8b3e12c80ec8d332dddf49b142922...  4967LYL
6  0fa4112b0123f7dfdf03baf1dc8e8d8d87dec16853c36c...  0476MNN
7  0f6bc9cea7f38650e5690d87a900fff67e3119052472e5...  2377JZG
8  0f5497916aaefa1109b4d90c26d6d316c80ec0b8a9365e...  4436MLF
9  0f5497916aaefa1109b4d90c26d6d316c80ec0b8a9365e...  6801HMT


In [5]:
class PlatesDataset(Dataset):
    def __init__(self, df, img_dir, transform=None, max_len=10):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.max_len = max_len


    def __len__(self):
        return len(self.df)
    
    
    def text_to_labels(self, text):
        labels = []
        for ch in text:
            if ch in char2idx:
                labels.append(char2idx[ch])
        return labels
    
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['filename'])
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            # imagen corrupta -> devolver zeros
            img = np.zeros((IMG_H, IMG_W), dtype=np.uint8)
        # resize manteniendo ratio y rellenando
        h, w = img.shape
        new_h = IMG_H
        new_w = int(w * (IMG_H / h))
        new_w = min(new_w, IMG_W)
        img = cv2.resize(img, (new_w, new_h))
        if new_w < IMG_W:
            pad = np.full((IMG_H, IMG_W - new_w), 255, dtype=np.uint8)
            img = np.concatenate([img, pad], axis=1)
        img = img.astype(np.float32) / 255.0
        img = img[np.newaxis, ...] # (1, H, W)
        
        
        labels = self.text_to_labels(row['text'])
        labels = np.array(labels, dtype=np.int32)
        return torch.tensor(img).float(), torch.tensor(labels).int(), len(labels)

In [6]:
class CRNN(nn.Module):
    def __init__(self, num_classes):
        super(CRNN, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.lstm = nn.LSTM(64*16, 128, num_layers=2, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.cnn(x)
        b, c, h, w = x.size()
        x = x.permute(0, 3, 1, 2).contiguous().view(b, w, c*h)
        x, _ = self.lstm(x)
        x = self.fc(x)
        return x

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CRNN(NUM_CLASSES).to(device)

In [7]:
from torch.nn.utils.rnn import pad_sequence


def collate_fn(batch):
    imgs, labels, lengths = zip(*batch)
    imgs = torch.stack(imgs)
    labels_concat = torch.cat([l for l in labels])
    lengths = torch.tensor(lengths, dtype=torch.long)
    return imgs, labels_concat, lengths

In [8]:
from torch.nn.utils.rnn import pad_sequence


def collate_fn(batch):
    imgs, labels, lengths = zip(*batch)
    imgs = torch.stack(imgs)
    labels_concat = torch.cat([l for l in labels])
    lengths = torch.tensor(lengths, dtype=torch.long)
    return imgs, labels_concat, lengths

In [9]:
train_df = df.sample(frac=0.9, random_state=42).reset_index(drop=True)
val_df = df.drop(train_df.index).reset_index(drop=True)


train_ds = PlatesDataset(train_df, IMG_DIR)
val_ds = PlatesDataset(val_df, IMG_DIR)


train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)


print(f"Train samples: {len(train_df)}, Val samples: {len(val_df)}")

Train samples: 65, Val samples: 7


In [10]:
class CRNN(nn.Module):
    def __init__(self, num_classes):
        super(CRNN, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d((2,1), (2,1)),  # conserva ancho
            nn.Conv2d(256, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d((2,1), (2,1))
        )

        # 🔹 Cambiamos el input_size a 1024 para que coincida con la salida real del CNN
        self.rnn = nn.LSTM(1024, 256, num_layers=2, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.cnn(x)
        b, c, h, w = x.size()
        x = x.view(b, c*h, w).permute(0, 2, 1)  # (batch, width, features)
        x, _ = self.rnn(x)
        x = self.fc(x)
        return x


In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CRNN(NUM_CLASSES).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.CTCLoss(blank=0, zero_infinity=True)


# helper para calcular input_lengths


def train_epoch(loader):
    model.train()
    running_loss = 0.0
    for imgs, labels_concat, lengths in tqdm(loader):
        imgs = imgs.to(device)
        labels_concat = labels_concat.to(device)
        batch = imgs.size(0)


        optimizer.zero_grad()
        outputs = model(imgs) # (B, W, C)
        outputs = outputs.log_softmax(2)
        outputs = outputs.permute(1,0,2) # (W, B, C) para CTCLoss
        
        
        input_lengths = torch.full(size=(batch,), fill_value=outputs.size(0), dtype=torch.long).to(device)
        target_lengths = lengths.to(device)
        
        
        loss = criterion(outputs, labels_concat, input_lengths, target_lengths)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(loader)




def val_epoch(loader):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for imgs, labels_concat, lengths in tqdm(loader):
            imgs = imgs.to(device)
            labels_concat = labels_concat.to(device)
            batch = imgs.size(0)
            outputs = model(imgs)
            outputs = outputs.log_softmax(2)
            outputs = outputs.permute(1,0,2)
            input_lengths = torch.full(size=(batch,), fill_value=outputs.size(0), dtype=torch.long).to(device)
            target_lengths = lengths.to(device)
            loss = criterion(outputs, labels_concat, input_lengths, target_lengths)
            running_loss += loss.item()
    return running_loss / len(loader)

In [12]:
best_val = float('inf')

for epoch in range(1, NUM_EPOCHS + 1):
    tr_loss = train_epoch(train_loader)
    val_loss = val_epoch(val_loader)
    print(f"Epoch {epoch}/{NUM_EPOCHS} - train_loss: {tr_loss:.4f} - val_loss: {val_loss:.4f}")

    # Guardar el mejor modelo
    if val_loss < best_val:
        best_val = val_loss
        torch.save(model, "/kaggle/working/ocr_ctc_full.pt")
        print("✅ Guardado mejor modelo como ocr_ctc_full.pt")


100%|██████████| 1/1 [00:00<00:00,  4.54it/s]


Epoch 1/30 - train_loss: 9.1616 - val_loss: 5.1755
✅ Guardado mejor modelo como ocr_ctc_full.pt


100%|██████████| 1/1 [00:00<00:00,  4.88it/s]


Epoch 2/30 - train_loss: 4.1725 - val_loss: 3.8328
✅ Guardado mejor modelo como ocr_ctc_full.pt


100%|██████████| 1/1 [00:00<00:00,  5.07it/s]


...

Epoch 28/30 - train_loss: 3.2250 - val_loss: 3.1397
✅ Guardado mejor modelo como ocr_ctc_full.pt


100%|██████████| 1/1 [00:00<00:00,  4.80it/s]


Epoch 29/30 - train_loss: 2.9985 - val_loss: 3.0971
✅ Guardado mejor modelo como ocr_ctc_full.pt


100%|██████████| 1/1 [00:00<00:00,  4.65it/s]

Epoch 30/30 - train_loss: 3.2266 - val_loss: 3.0749
✅ Guardado mejor modelo como ocr_ctc_full.pt
